In [1]:
!pip install langchain openai chromadb pymupdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
 

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [5]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from IPython.display import display, Markdown

In [6]:
load_dotenv("secrets.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key is not None, "OPENAI_API_KEY가 secrets.env에 없습니다."
os.environ["OPENAI_API_KEY"] = openai_api_key

In [7]:
pdf_url = "https://arxiv.org/pdf/2005.11401.pdf"
!wget -O paper.pdf $pdf_url

loader = PyMuPDFLoader("paper.pdf")
raw_documents = loader.load()

--2025-04-24 15:08:37--  https://arxiv.org/pdf/2005.11401.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2005.11401 [following]
--2025-04-24 15:08:37--  http://arxiv.org/pdf/2005.11401
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 885323 (865K) [application/pdf]
Saving to: ‘paper.pdf’

paper.pdf           100%[===================>] 864.57K  --.-KB/s    in 0.07s   

2025-04-24 15:08:37 (12.7 MB/s) - ‘paper.pdf’ saved [885323/885323]



In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "]
)
documents = text_splitter.split_documents(raw_documents)

In [9]:
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

<ipython-input-9-a961904a60e5>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [17]:
retrieved = retriever.get_relevant_documents("summarize the academic paper")
context = "\n\n".join([doc.page_content for doc in retrieved])

<ipython-input-17-4ce312eb8b51>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved = retriever.get_relevant_documents("summarize the academic paper")


In [23]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 🧠 LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 📝 프롬프트 템플릿 구성
template = """
You are a helpful research assistant.
Here is the content of an academic paper:

{context}

Please summarize the paper in ko-KR with the following structure:
1. Problem Statement
2. Proposed Method
3. Main Results
4. Conclusion and Impact
"""

prompt = PromptTemplate(input_variables=["context"], template=template)

# 🔗 LLMChain으로 연결
chain = LLMChain(llm=llm, prompt=prompt)

# 🚀 실행
result = chain.invoke({"context": context})
display(Markdown(f"### 📝 요약 결과\n\n{result['text']}"))

### 📝 요약 결과

### 1. 문제 진술
이 논문은 BART 모델의 메모리 구조를 탐구하며, 파라메트릭 메모리와 비파라메트릭 메모리가 어떻게 상호작용하여 텍스트 생성을 지원하는지를 조사합니다. 특히, BART가 특정 지식을 어떻게 저장하고 활용하는지를 보여주는 예시를 통해 이론적 근거를 제시합니다.

### 2. 제안된 방법
저자들은 BART 모델을 사용하여 부분적으로 주어진 텍스트를 완성하는 실험을 수행했습니다. 예를 들어, "The Sun"이라는 부분적 디코딩을 입력으로 제공했을 때, BART는 "The Sun Also Rises"라는 소설 제목을 생성합니다. 이 과정에서 비파라메트릭 메모리가 파라메트릭 메모리에서 특정 지식을 끌어내는 데 도움을 주는 방식으로 작동함을 보여줍니다.

### 3. 주요 결과
FEVER 데이터셋을 사용한 3-way 분류 실험에서 RAG 모델은 최첨단 모델들과 비교했을 때 4.3% 이내의 성능을 보였습니다. 이들 최첨단 모델은 복잡한 파이프라인 시스템과 도메인 특화 아키텍처를 필요로 하며, 중간 검색 감독을 통해 훈련됩니다. 반면, RAG는 이러한 감독 없이도 유사한 성능을 달성했습니다.

### 4. 결론 및 영향
이 연구는 BART 모델의 메모리 구조와 텍스트 생성 과정에서의 상호작용을 이해하는 데 기여합니다. 또한, RAG 모델의 성능이 복잡한 시스템 없이도 경쟁력을 가질 수 있음을 보여주어, 향후 AI 모델 개발에 있어 효율성과 단순성을 강조하는 방향으로의 연구를 촉진할 수 있습니다.